# MRV Emissions Exploration

_Author: Robert Dibble_

The European Union requires ships larger than 5000 tons that dock in the EU to declare their emissions to the MRV system, the data of which is made available publicly. This notebook aims to explore the dataset and investigate any modelling opportunities.

## Initialise

### Import libraries

In [ ]:
from datavizml.exploratorydataanalysis import ExploratoryDataAnalysis
import numpy as np
import os
import pandas as pd
import utils
import warnings
import unittest

# run unit tests
_ = unittest.TextTestRunner(verbosity=2).run(
    unittest.TestLoader().loadTestsFromModule(utils)
)

### Load raw data

Sourced from [EU-MRV system](https://mrv.emsa.europa.eu/#public/emission-report)

_N.B. Brexit is not reflected until 2021 (incl) onwards_

In [ ]:
# initialise empty list to store each year's dataframe in
yearly_data = []

# extract all filenames in 'data' folder
filename_all = os.listdir("data")

# loop through all files in folder
for i, filename in enumerate(filename_all):
    # print progress through folder
    print(f"Loading {i+1} of {len(filename_all)}: {filename}")

    # suppress irrelevant warning "Workbook contains no default style, apply openpyxl's default"
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        # load dataframe
        df_temp = pd.read_excel(
            os.path.join("data", filename),
            header=2,
            na_values=[
                "Division by zero!"
            ],  # add additional values as observed throughout development process
        )

        # rename column as name changed between 2019 and 2020
        df_temp.rename(
            columns={
                "Annual Total time spent at sea [hours]": "Annual Time spent at sea [hours]"
            },
            inplace=True,
        )

        # add current dataframe to list of other year's dataframes
        yearly_data.append(df_temp)

# combine data from each year into one dataframe
raw_df = pd.concat(yearly_data, ignore_index=True)

In [ ]:
# check that no vessel is duplicated within a year
imo_year_count = raw_df[["IMO Number", "Reporting Period"]].value_counts()
assert (imo_year_count == 1).all()

In [ ]:
# downselect columns of interest for brevity and set dtype to minimise memory use
selected_columns_dtype = {
    "IMO Number": int,
    "Ship type": "category",
    "Reporting Period": int,
    "Technical efficiency": object,
    "Total fuel consumption [m tonnes]": float,
    "Fuel consumptions assigned to On laden [m tonnes]": float,
    "Total CO₂ emissions [m tonnes]": float,
    "CO₂ emissions assigned to Passenger transport [m tonnes]": float,
    "CO₂ emissions assigned to Freight transport [m tonnes]": float,
    "CO₂ emissions assigned to On laden [m tonnes]": float,
    "Annual Time spent at sea [hours]": float,
    "Annual average Fuel consumption per distance [kg / n mile]": float,
    "Annual average CO₂ emissions per distance [kg CO₂ / n mile]": float,
}
main_df = raw_df[selected_columns_dtype.keys()].astype(selected_columns_dtype)

### Process _Technical efficiency_ data

_Technical efficiency_ is the amount of CO₂ release per tonne of capacity per nautical mile traversed. This data is reported as a string in the format `'<calculation type> (<numerical value> gCO₂/t·nm)'` and subsequently needs to be preocessed to separate these 2 pieces of information.

The preferred method to calulcate this is to use the Energy Efficiency Design Index (EEDI) equation:
$$\dfrac{\Sigma_{i=1}^{N} \left( P_i \times SFC_i \times CF_i \right)}{Capacity \times Speed}$$
where:
- $N$ is the number of engines
- $P_i$ is the power of the $i^{th}$ engine
- $SFC_i$ is the specific fuel consumption of the $i^{th}$ engine
- $CF_i$ is the carbon emission factor of the fuel of the $i^{th}$ engine.

Alternatively, EIV (Estimated Index Value) is calculated using the following assumptions for main engines (ME) and auxilliary engines (AE):
- The carbon emission factor ($CF$) is constant for all engines, $CF_{ME}$ = $CF_{AE}$ = 3.1144 g CO₂/g fuel.
- The specific fuel consumption ($SFC$) for all ship types is constant for all main engines, $SFC_{ME}$ = 190 g/kWh.
- The specific fuel consumption for all ship types is constant for all auxiliary engines, $SFC_{AE}$ = 215 g/kWh.
- Capacity is defined as 70% of deadweight for containerships and 100% of deadweight for other ship types.

Sources:
1. EEIV: [MAN Energy Solutions](https://www.man-es.com/docs/default-source/document-sync/eedi-eng.pdf?sfvrsn=23fbab95_2)
1. EIV: [Transport and Environment office](https://www.transportenvironment.org/wp-content/uploads/2021/07/2015%2005%20CE_Delft_7E50_Estimated_Index_Values_of_New_Ships_DEF.pdf)


In [ ]:
# split column into list of both values
split_combined = main_df["Technical efficiency"].apply(
    lambda x: utils.tech_eff_extract(str(x))
)

# separate lists into individual columns in oringinal dataframe
split_separate = pd.DataFrame(split_combined.to_list()).astype(
    {0: "category", 1: float}
)
main_df[
    ["Technical efficiency type", "Technical efficiency value [gCO₂/t·nm]"]
] = split_separate

# drop unprocessed column
main_df.drop(columns=["Technical efficiency"], inplace=True)

### Calculate distance travelled, average speed and ship deadweight

In [ ]:
# calculate distance travelled

# calculate distance from both pairs of columns
dist_fuel_temp = (
    main_df["Total fuel consumption [m tonnes]"]
    * 1000
    / main_df["Annual average Fuel consumption per distance [kg / n mile]"]
)
dist_co2_temp = (
    main_df["Total CO₂ emissions [m tonnes]"]
    * 1000
    / main_df["Annual average CO₂ emissions per distance [kg CO₂ / n mile]"]
)

# average the two methods
dist_temp = (dist_fuel_temp + dist_co2_temp - 1) / 2

# assert the 2 methods agree with each other (to within 1%)
assert not ((abs(dist_fuel_temp - dist_co2_temp) / dist_temp - 1) > 0.01).any()

# save values to dataframe
main_df["Distance travelled [n mile]"] = dist_temp

In [ ]:
# calculate average speed
main_df["Average speed [knots]"] = (
    main_df["Distance travelled [n mile]"] / main_df["Annual Time spent at sea [hours]"]
).replace(np.inf, np.nan)

In [ ]:
# calculate ship deadweight

# calculate capacity as per EEDI calculation
cap_temp = (
    main_df["Total fuel consumption [m tonnes]"]
    * 1e6
    / main_df["Technical efficiency value [gCO₂/t·nm]"]
    / main_df["Distance travelled [n mile]"]
).replace(np.inf, np.nan)

# calculate deadweight from capacity using EIV assumption about container ships
cap_dwt_ratio = main_df["Ship type"].apply(
    lambda x: 0.7 if "container" in x.lower() else 1.0
)
main_df["Deadweight [m tonnes]"] = cap_temp / cap_dwt_ratio

## Data preprocessing to remove/reduce missing data and outliers

In [ ]:
# drop rows where ship has consumed no fuel and emitted no CO₂
main_df = main_df.loc[
    (main_df["Total fuel consumption [m tonnes]"] != 0)
    | (main_df["Total CO₂ emissions [m tonnes]"] != 0)
]

In [ ]:
# limit annual time at sea to the duration of a year
main_df["Annual Time spent at sea [hours]"].clip(upper=365 * 24, inplace=True)

In [ ]:
# remove outliers based on standard deviation of column values; either considering their raw values of their log10
main_df = utils.OutlierRemoval(
    sd_threshold=4,  # considering the size of the dataset, data points beyond the normally accepted 3SD would be expected
    linear_cols=[],
    log10_cols=[
        "Total fuel consumption [m tonnes]",
        "Fuel consumptions assigned to On laden [m tonnes]",
        "Total CO₂ emissions [m tonnes]",
        "CO₂ emissions assigned to Passenger transport [m tonnes]",
        "CO₂ emissions assigned to Freight transport [m tonnes]",
        "CO₂ emissions assigned to On laden [m tonnes]",
        "Annual Time spent at sea [hours]",
        "Annual average Fuel consumption per distance [kg / n mile]",
        "Annual average CO₂ emissions per distance [kg CO₂ / n mile]",
        "Technical efficiency value [gCO₂/t·nm]",
        "Distance travelled [n mile]",
        "Average speed [knots]",
        "Deadweight [m tonnes]",
    ],
).fit_transform(main_df)

In [ ]:
# calculate missing data proportion
missing_prop = main_df.isnull().mean()

In [ ]:
# drop columns with too much missing data
drop_cols = missing_prop.loc[missing_prop > 0.5].index
main_df.drop(columns=drop_cols, inplace=True)

## Exploratory data analysis

In [ ]:
eda = ExploratoryDataAnalysis(
    data=main_df.drop(columns="IMO Number"), ncols=6, axes_height=3.5
)
fig = eda()
fig.tight_layout()